In [2]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
!docker run --gpus all nvidia/cuda:9.0-base nvidia-smi

Mon Nov  4 07:10:57 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00005D4A:00:00.0 Off |                    0 |
| N/A   39C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           On   | 00007FAF:00:00.0 Off |                    0 |
| N/A   34C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
%load_ext dotenv

In [4]:
import os

In [5]:
!pwd
!ls -l ./

/datadrive01/amlSDKAzureChestXray/code
total 860
-rwxrwxrwx 1 login_VM01 login_VM01  22796 Jan 24 18:12 00_create_docker_image_sdk.ipynb
drwxrwxrwx 2 login_VM01 login_VM01   4096 Jan 15 21:26 aml_config
-rwxrwxrwx 1 login_VM01 login_VM01  19353 Jan 24 18:07 evaluate.ipynb
-rwxrwxrwx 1 login_VM01 login_VM01 486658 Jan 24 17:42 training.ipynb
-rwxrwxrwx 1 login_VM01 login_VM01 339667 Jan 23 19:36 training-no-gpu-detection.ipynb
drwxrwxrwx 6 login_VM01 login_VM01   4096 Jan 24 17:40 xray-scripts


In [6]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'kateyuan'
image_tag = "/chestxraynoaml"
image_version = '1.0.4'

Overwriting .env


In [7]:
%dotenv
dockerfile_name = 'dockerfile'+ '_' + os.getenv('image_version')

In [13]:
%%writefile ./../docker/$dockerfile_name

FROM nvidia/cuda:9.0-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh 
        
#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
# https://www.microsoft.com/net/download/linux-package-manager/ubuntu16-04/runtime-2.1.2
RUN wget -q https://packages.microsoft.com/config/ubuntu/16.04/packages-microsoft-prod.deb && \
    dpkg -i packages-microsoft-prod.deb && \ 
    apt-get install  -y --no-install-recommends apt-transport-https && \
    apt-get update && \
    apt-get install  -y --no-install-recommends aspnetcore-runtime-2.1  && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
# ENV NB_USER keras
# ENV NB_UID 1000

# RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
#     chown $NB_USER $CONDA_DIR -R && \
#     mkdir -p /src && \
#     chown $NB_USER /src

# RUN chmod -R ugo=rwx /

# USER $NB_USER

RUN mkdir -p /src 

ARG python_version=3.6

# tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow-gpu \
      tensorflow-tensorboard \
      matplotlib \
      numpy==1.14.5 && \
    conda install \
      bcolz \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook=5.6.0 \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      requests \
      scikit-learn \
      six \
      tqdm && \
    conda install -c conda-forge imageio && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    pip install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl \
      torchvision && \
    conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile_1.0.4


In [31]:
# %%writefile ./../docker/$dockerfile_name

# FROM nvidia/cuda:9.0-cudnn7-devel
# #FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

# MAINTAINER George Iordanescu <ghiordan@microsoft.com>
# #based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# # Install system packages
# RUN apt-get update && apt-get install -y --no-install-recommends \
#       apt-utils \
#       bzip2 \
#       g++ \
#       git \
#       graphviz \
#       libgl1-mesa-glx \
#       libhdf5-dev \
#       openmpi-bin \
#       wget \
#       ssh \
#       rsync && \
#     rm -rf /var/lib/apt/lists/*

# # Install conda
# ENV CONDA_DIR /opt/conda
# ENV PATH $CONDA_DIR/bin:$PATH

# RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
#     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
#     /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
#     rm Miniconda3-4.2.12-Linux-x86_64.sh && \
#     echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh 
        
# #https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# # https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
# # https://www.microsoft.com/net/download/linux-package-manager/ubuntu16-04/runtime-2.1.2
# RUN wget -q https://packages.microsoft.com/config/ubuntu/16.04/packages-microsoft-prod.deb && \
#     dpkg -i packages-microsoft-prod.deb && \ 
#     apt-get install  -y --no-install-recommends apt-transport-https && \
#     apt-get update && \
#     apt-get install  -y --no-install-recommends aspnetcore-runtime-2.1  && \
#     mkdir /tmp/azcopy && \
#     wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
#     tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
#     /tmp/azcopy/install.sh && \
#     rm -rf /tmp/azcopy

# # Install Python packages and keras
# ENV NB_USER keras
# ENV NB_UID 1000

# RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
#     chown $NB_USER $CONDA_DIR -R && \
#     mkdir -p /src && \
#     chown $NB_USER /src

# USER $NB_USER

# ARG python_version=3.6

# # tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
# RUN conda install -y python=${python_version} && \
#     pip install --upgrade pip && \
#     conda clean -yt     

# #ADD theanorc /home/keras/.theanorc

# COPY ./../../../data /data 

# ENV PYTHONPATH='/src/:$PYTHONPATH'

# WORKDIR /src

# EXPOSE 8888

# # CMD jupyter notebook --port=8888 --ip=0.0.0.0


In [14]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':' + os.getenv('image_version')
docker_file_location = os.path.join(*(['.', '..','docker', dockerfile_name]))
working_path = '.'

docker_image_name
docker_file_location

'kateyuan/chestxraynoaml:1.0.4'

'./../docker/dockerfile_1.0.4'

In [15]:
!mkdir -p ./docker_history
!cp $docker_file_location ./docker_history/

In [16]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t kateyuan/chestxraynoaml:1.0.4 -f ./../docker/dockerfile_1.0.4 .
Sending build context to Docker daemon  8.049MB
Step 1/13 : FROM nvidia/cuda:9.0-cudnn7-devel
 ---> afc5ab1e9a0d
Step 2/13 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> c3bbbb73b78f
Step 3/13 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 5cd98d674a82
Step 4/13 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> 461fbd6fc6dd
Step 5/13 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> 7a90bf36fa97
Step 6/13 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum 

  Stored in directory: /root/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6


  Stored in directory: /root/.cache/pip/wheels/90/db/f8/2c3101f72ef1ad434e4662853174126ce30201a3e163dcbeca
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx termcolor absl-py gast html5lib toolz
Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    backcall:          0.1.0-py36_0          
    bcolz:             1.2.1-py36h04863e7_0  
    blas:              1.0-mkl               
    bleach:            3.1.0-py36_0          
    blosc:             1.15.0-hd408876_0     
    bokeh:             1.0.4-py36_0          
    bzip2:             1.0.6-h14c3975_5      
    ca

blas-1.0-mkl.t 100% |###############################| Time: 0:00:00   9.66 MB/s
conda-env-2.6. 100% |###############################| Time: 0:00:00   6.79 MB/s
intel-openmp-2 100% |###############################| Time: 0:00:00  63.24 MB/s
libgcc-ng-8.2. 100% |###############################| Time: 0:00:00  70.30 MB/s
libgfortran-ng 100% |###############################| Time: 0:00:00  32.84 MB/s
libstdcxx-ng-8 100% |###############################| Time: 0:00:00  68.78 MB/s
bzip2-1.0.6-h1 100% |###############################| Time: 0:00:00  65.35 MB/s
expat-2.2.6-he 100% |###############################| Time: 0:00:00  71.46 MB/s
freeglut-3.0.0 100% |###############################| Time: 0:00:00  68.85 MB/s
fribidi-1.0.5- 100% |###############################| Time: 0:00:00  66.39 MB/s
gmp-6.1.2-h6c8 100% |###############################| Time: 0:00:00  70.83 MB/s
graphite2-1.3. 100% |###############################| Time: 0:00:00  64.03 MB/s
icu-58.2-h9c2b 100% |###################

scipy-1.2.0-py 100% |###############################| Time: 0:00:00  65.13 MB/s
dask-1.0.0-py3 100% |###############################| Time: 0:00:00   9.75 MB/s
opencv-3.4.2-p 100% |###############################| Time: 0:00:00  27.48 MB/s
pytables-3.4.4 100% |###############################| Time: 0:00:00  36.96 MB/s
scikit-learn-0 100% |###############################| Time: 0:00:00  65.91 MB/s
bcolz-1.2.1-py 100% |###############################| Time: 0:00:00  69.26 MB/s
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - imageio


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    imageio-2.4.1              |        py36_1000         3.4 MB  conda-forge
    certifi-2018.11.29         |        py36_1000         145 KB  conda-forge
   

  Running setup.py develop for Keras
  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-gxegytkg
  Stored in directory: /tmp/pip-ephem-wheel-cache-78aei9_y/wheels/e6/02/ad/5e8e1a5824af71082e2260fe7e2eaa1b745c34706e6ff0a14b
Successfully built Keras
  Cloning https://github.com/aleju/imgaug to /tmp/pip-req-build-r9c5ilpx
  Stored in directory: /tmp/pip-ephem-wheel-cache-tapkh2zt/wheels/9c/f6/aa/41dcf2f29cc1de1da4ad840ef5393514bead64ac9e644260ff
Successfully built imgaug
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-e8vmquze
  Stored in directory: /tmp/pip-ephem-wheel-cache-dnuashyg/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
zeromq-4.2.5-hf484d3e_1.tar.bz2              567 KB
pickleshare-0.7.5-py36_0.tar.bz2              13 KB
freetype-2.9.1-h8a8886c_1.tar.bz2            822 KB

matplotlib-3.0.2-py36h5429711_0.tar.bz2      6.5 MB
chardet-3.0.4-py36_1.tar.bz2                 189 KB
terminado-0.8.1-py36_1.tar.bz2                21 KB
python-3.6.2-0.tar.bz2                      16.5 MB
testpath-0.4.2-py36_0.tar.bz2                 91 KB
six-1.10.0-py36_0.tar.bz2                     19 KB
mkl_fft-1.0.10-py36ha843d7b_0.tar.bz2        170 KB
cryptography-1.8.1-py36_0.tar.bz2            846 KB
lzo-2.10-h49e0be7_2.tar.bz2                  313 KB
olefile-0.46-py36_0.tar.bz2                   48 KB
mkl_random-1.0.2-py36hd81dba3_0.tar.bz2      407 KB
prompt_toolkit-2.0.7-py36_0.tar.bz2          482 KB

---------------------------------------------------
Total:                                     652.8 MB

Removed zeromq-4.2.5-hf484d3e_1.tar.bz2
Removed pickleshare-0.7.5-py36_0.tar.bz2
Removed freetype-2.9.1-h8a8886c_1.tar.bz2
Removed pysocks-1.6.8-py36_0.tar.bz2
Removed pillow-5.4.1-py36h34e0f95_0.tar.bz2
Removed libedit-3.1-heed3624_0.tar.bz2
Removed sortedcontainers-2.

In [17]:
!docker push $docker_image_name

The push refers to repository [docker.io/kateyuan/chestxraynoaml]

440bd7cf: Preparing 
17378786: Preparing 
3e0e2339: Preparing 
fde91465: Preparing 
964f5e93: Preparing 
39f3990e: Preparing 
347e5346: Preparing 
9e5e9701: Preparing 
51016399: Preparing 
b658aaf0: Preparing 
a9c7c79e: Preparing 
a8049aa6: Preparing 
9c0f8a0b: Preparing 
8ccd260b: Preparing 


40bd7cf: Pushing  1.705GB/5.86GBPushing  460.7MB/5.86GBPushing  1.051GB/5.86GBPushing  1.278GB/5.86GBPushing  1.354GB/5.86GB40bd7cf: Pushing  3.544GB/5.86GBPushing  2.116GB/5.86GBPushing  2.323GB/5.86GBPushing  2.588GB/5.86GBPushing  2.989GB/5.86GBPushing  3.282GB/5.86GB40bd7cf: Pushing  5.415GB/5.86GBPushing  3.828GB/5.86GBPushing  3.949GB/5.86GBPushing  4.108GB/5.86GBPushing  4.998GB/5.86GB

40bd7cf: Pushed   5.928GB/5.86GBPushing  5.606GB/5.86GBPushing  5.761GB/5.86GB1.0.4: digest: sha256:51ed22159c4f70c4cb0770b09c716c1949b531c5be524bd96630bad2fae10354 size: 3475


In [18]:
!echo nvidia-docker run -i -t -p 10002:8888 -v '$(pwd)':/local_dir:rw $docker_image_name /bin/bash -c '"/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip='*' --port=8888 --no-browser --allow-root"'

nvidia-docker run -i -t -p 10002:8888 -v $(pwd):/local_dir:rw kateyuan/chestxraynoaml:1.0.3 /bin/bash -c "/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip=* --port=8888 --no-browser --allow-root"


In [19]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 389050 bytes to 00_create_docker_image.html
